#### LangSmith 기본 예제


##### 1) 라이브러리 설치

In [ ]:
# poetry add langsmith

##### 2) OpenAI 인증키 설정

In [ ]:
import os
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

##### LangSmith와 LangChain을 활용한 기본 로깅 예제

In [ ]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langsmith import traceable

# LangSmith API Key 설정
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")  # LangSmith 활성화
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # API Key 불러오기
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")   # 프로젝트 이름 설정
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")   # EndPoint 설정

# LLM 모델 설정 (OpenAI 사용)
#llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# LangSmith로 실행 추적
@traceable(run_type="chain", name="Simple_Chain")
def ask_question(question: str):

    # 개별 메시지 템플릿 정의
    system_message = SystemMessagePromptTemplate.from_template(
        "당신은 유용한 AI 비서입니다."
    )
    user_message = HumanMessagePromptTemplate.from_template(
        "{question}"
    )

    chat_prompt = ChatPromptTemplate.from_messages([
        system_message,
        user_message,
    ])
    
    messages = chat_prompt.format_messages(question=question)
    response = llm.invoke(messages)

    return response.content

# 테스트 실행
question = "LangGraph와 LangChain의 차이점은 무엇인가요?"
answer = ask_question(question)

# 결과 출력
print("\n🔹 [AI 답변]:")
print(answer)

